In [ ]:
from dotenv import load_dotenv
import os
from groq import Groq
# Carregandos os dados da .env
load_dotenv()

True

In [ ]:
import os

from groq import Groq
# Inicializando o cliente Groq com a chave da API
client = Groq(
    api_key=os.environ.get("GROQ_API"),
)
# Criando uma solicitação de conclusão de chat
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explain the importance of fast language models",
        }
    ],
    model="llama-3.3-70b-versatile",
)
# Exibindo o conteúdo da resposta
print(chat_completion.choices[0].message.content)


Fast language models are crucial in today's technology landscape, and their importance can be summarized as follows:

1. **Improved User Experience**: Fast language models can process and respond to user queries quickly, providing an enhanced user experience. This is particularly important in applications like chatbots, virtual assistants, and language translation software, where speed and accuracy are critical.
2. **Real-Time Applications**: Fast language models can handle real-time applications, such as speech recognition, sentiment analysis, and content moderation, which require immediate processing and response.
3. **Efficient Data Processing**: Fast language models can process large amounts of data quickly, making them ideal for applications like text analysis, data mining, and information retrieval.
4. **Reduced Latency**: Fast language models can reduce latency, which is the time it takes for a model to process a request and respond. This is particularly important in application

In [ ]:
# Groq Agent Class
class Agent:
    # Construtor de classe para inicializar o cliente Groq e o sistema
    def __init__(self, client: Groq, system: str = "") -> None:
        self.client = client
        self.system = system
        self.messages: list = []
        if self.system:
            self.messages.append({"role": "system", "content": system})
# Método para adicionar uma mensagem do usuário e obter uma resposta do modelo
    def __call__(self, message=""):
        if message:
            self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result
# Método para executar a solicitação de conclusão de chat
    def execute(self):
        completion = client.chat.completions.create(
            model="llama-3.3-70b-versatile", messages=self.messages
        )
        return completion.choices[0].message.content

In [ ]:
# Definindo o sistema de prompt para o agente
system_prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer.
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary.

estimate_swimming_strokes:
e.g. estimate_swimming_strokes: height=1.9, level='elite', pool_length=50
Estimates the number of strokes (one arm) to cross a pool. 
Parameters:
- height: The swimmer's height in meters.
- level: The swimmer's skill level ('beginner', 'intermediate', or 'elite').
- pool_length: The length of the pool in meters (e.g., 25 or 50).

Example session:

Question: Quantas braçadas eu preciso dar na piscina sendo um nadador de elite com 1,75m?
Thought: A pergunta pede uma estimativa de braçadas para um nadador de elite com 1,75m. A ferramenta `estimate_swimming_strokes` é a ideal para isso. A pergunta não especifica o tamanho da piscina, então vou assumir o padrão olímpico, que é de 50 metros, pois é o mais comum para o nível 'elite'. Vou usar os parâmetros height=1.75, level='elite', e pool_length=50.
Action: estimate_swimming_strokes: height=1.75, level='elite', pool_length=50
PAUSE

You will be called again with this:

Observation: 37

Thought: A ferramenta retornou uma estimativa de 37 braçadas. Esta é a resposta final para a pergunta. Agora vou formatá-la de forma clara para o usuário.
Answer: Para um nadador de elite com 1,75m de altura, a estimativa é de aproximadamente 37 braçadas para atravessar uma piscina olímpica de 50 metros.

Now it's your turn:
""".strip()

In [ ]:
# ferramenta = são as funções que o agente pode chamar
import math

def calculate(operation: str) -> float: # operação de cálculo 
    try:
        return eval(operation) # avalia a expressão matemática
    except Exception as e:
        return f"Erro no cálculo: {e}" #  retorna erro se houver um problema na avaliação

def estimate_swimming_strokes(height: float, level: str, pool_length: int) -> int:
    efficiency_factors = { # fatores de eficiência para diferentes níveis de nadadores
        'beginner': 1.2,
        'intermediate': 1.0,
        'elite': 0.85
    }
    # Verifica se o nível é válido
    level_factor = efficiency_factors.get(level.lower(), 1.0)
    # Verifica se a altura é válida
    distance_per_stroke = height * 0.95 # distancia da braçada em metros, 95% da altura do nadador
    # Verifica se a distância por braçada é zero
    if distance_per_stroke == 0:
        return 0
    # Calcula o número de braçadas necessárias    
    strokes = (pool_length / distance_per_stroke) * level_factor # quantas braçadas são necessárias
    # Verifica se o número de braçadas é negativo
    return math.ceil(strokes) # arrendondando para cima para garantir que cubra a distância

In [ ]:
# Criando uma instância do agente com o cliente Groq e o sistema de prompt
michael_phelps = Agent(client, system_prompt) 

In [ ]:
# Exemplo de uso do agente para estimar o número de braçadas
result =  michael_phelps("Quantas braçadas eu preciso dar na piscina sendo um nadador de elite com 1,75m?")
print(result) 

Thought: A pergunta pede uma estimativa de braçadas para um nadador de elite com 1,75m. A ferramenta `estimate_swimming_strokes` é a ideal para isso. A pergunta não especifica o tamanho da piscina, então vou assumir o padrão olímpico, que é de 50 metros, pois é o mais comum para o nível 'elite'. Vou usar os parâmetros height=1.75, level='elite', e pool_length=50.
Action: estimate_swimming_strokes: height=1.75, level='elite', pool_length=50
PAUSE


In [ ]:
michael_phelps.messages

[{'role': 'system',
  'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer.\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary.\n\nestimate_swimming_strokes:\ne.g. estimate_swimming_strokes: height=1.9, level='elite', pool_length=50\nEstimates the number of strokes (one arm) to cross a pool. \nParameters:\n- height: The swimmer's height in meters.\n- level: The swimmer's skill level ('beginner', 'intermediate', or 'elite').\n- pool_length: The length of the pool in meters (e.g., 25 or 50).\n\nExample session:\n\nQuestion: Quantas braçadas eu preciso dar na piscina sendo um nadador de eli

In [ ]:
# Exemplo de uso do agente para calcular uma expressão matemática
result = michael_phelps()
print(result)

In [ ]:
#  Exemplo de uso da função estimate_swimming_strokes diretamente
observation = estimate_swimming_strokes(height=1.75, level='elite', pool_length=50)
print(observation)

26


In [ ]:
# Continuando o loop do agente com a observação obtida
next_prompt = f"Observation: {observation}"
result = michael_phelps(next_prompt)
print(result)

Thought: A ferramenta retornou uma estimativa de 26 braçadas. Esta é a resposta final para a pergunta. Agora vou formatá-la de forma clara para o usuário.
Answer: Para um nadador de elite com 1,75m de altura, a estimativa é de aproximadamente 26 braçadas para atravessar uma piscina olímpica de 50 metros.


In [73]:
michael_phelps.messages

[{'role': 'system',
  'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer.\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary.\n\nestimate_swimming_strokes:\ne.g. estimate_swimming_strokes: height=1.9, level='elite', pool_length=50\nEstimates the number of strokes (one arm) to cross a pool. \nParameters:\n- height: The swimmer's height in meters.\n- level: The swimmer's skill level ('beginner', 'intermediate', or 'elite').\n- pool_length: The length of the pool in meters (e.g., 25 or 50).\n\nExample session:\n\nQuestion: Quantas braçadas eu preciso dar na piscina sendo um nadador de eli

In [ ]:
import re 
def agent_loop(max_iterations, query): # função para executar o loop do agente
    agent = Agent(client, system_prompt)
    tools = ['calculate', 'estimate_swimming_strokes']
    next_prompt = f"N :{query}"
    i = 0

    while i < max_iterations:
        i += 1
        result = agent(next_prompt)
        print(f"Saída: {result}")
        
        if "PAUSE" in result and "Action" in result: # verifica se há uma ação a ser executada
            action_match = re.search(r"Action: ([a-zA-Z_]+):\s*(.*)", result, re.IGNORECASE) # mudei o regex lidar melhor com os argumentos
            
            if action_match: # encontrou uma ação
                chosen_tool, arg_string = action_match.groups()
                
                if chosen_tool in tools: # verifica se a ferramenta está disponível
                    function_call = f"{chosen_tool}({arg_string})"
                    print(f"Executando ação: {function_call}\n")
                    try:
                        result_tool = eval(function_call) # executa a função correspondente
                        next_prompt = f"Observation: {result_tool}"
                    except Exception as e: # captura erros na execução da ferramenta
                        next_prompt = f"Observation: Erro ao executar a ferramenta - {e}"
                else: # a ferramenta não está disponível
                    next_prompt = f"Observation: Ferramenta '{chosen_tool}' não encontrada."
            else: # não encontrou uma ação válida
                next_prompt = "Observation: Não foi possível analisar a Ação."

            print(f"--- Próximo Input para o Agente ---\n{next_prompt}\n")
            continue 

        if "Answer" in result:
            print("Missão concluída!")
            break


In [ ]:
agent_loop( 
    max_iterations=5, # defini o número de interações como 5 por ser  razoável para evitar loops infinitos
    query="Quantas braçadas eu preciso dar na piscina sendo um nadador de elite com 1,75m?"
)

Saída: Thought: A pergunta pede uma estimativa de braçadas para um nadador de elite com 1,75m. A ferramenta `estimate_swimming_strokes` é a ideal para isso. A pergunta não especifica o tamanho da piscina, então vou assumir o padrão olímpico, que é de 50 metros, pois é o mais comum para o nível 'elite'. Vou usar os parâmetros height=1.75, level='elite', e pool_length=50.

Action: estimate_swimming_strokes: height=1.75, level='elite', pool_length=50

PAUSE
Executando ação: estimate_swimming_strokes(height=1.75, level='elite', pool_length=50)

--- Próximo Input para o Agente ---
Observation: 26

Saída: Thought: A ferramenta retornou uma estimativa de 26 braçadas. Esta é a resposta final para a pergunta. Agora vou formatá-la de forma clara para o usuário.

Answer: Para um nadador de elite com 1,75m de altura, a estimativa é de aproximadamente 26 braçadas para atravessar uma piscina olímpica de 50 metros.
Missão concluída!
